# 多技能对话

多技能对话系统旨在建立一个开放域的多轮对话系统，能自然地融合多个对话技能，比如知识对话、推荐对话等，使得机器可以流畅自然地与人进行语言交互，从而有效地提升用户体验。

参考[2021语言与智能技术竞赛：多技能对话](https://aistudio.baidu.com/aistudio/competition/detail/67)基线

In [1]:
# 安装paddlenlp最新版本
!pip install --upgrade paddlenlp -i https://pypi.org/simple

%cd multi-skill_dialogue/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.4 kB/s eta 0:00:0000:0100:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.3/362.3 kB 28.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.2 kB/s eta 0:00:0000:0100:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 19.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 15.0 kB/s eta 0:00:00a 0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 23.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 28.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 16.1/29.3 MB 12.1 kB/s eta 0:18:13
ERROR: Exception:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pip/_vendor/urllib3/response.py", line 435, in _error_catcher
    yield


In [ ]:
%cd multi-skill_dialogue/

/home/aistudio/multi-skill_dialogue


In [ ]:
%pwd

'/home/aistudio/multi-skill_dialogue'

In [ ]:
%cd ./tools/

/home/aistudio/multi-skill_dialogue/tools


## 多技能对话基线

多技能对话比赛提供了多个子数据集，包含知识对话、推荐对话、画像对话和其他多种类型的对话数据集。基线采用UnifiedTransformer模型，模型的的输入除了数据token及`[CLS]`、`[SEP]`等special token之外，还有用于区别不同对话技能的special token。

![模型输入](https://ai-studio-static-online.cdn.bcebos.com/24d697df544c4299a679e04e2d3b1442fdf17a14981e454e8a2de5c7acea8051)

### Step1：数据预处理

由于多技能对话比赛的[数据集](https://aistudio.baidu.com/aistudio/competition/detail/67)**数量多且数据规模大**，并且数据集之间**格式不同**，所以需要使用脚本对数据集进行预处理，同时将数据转化成id化的数据。

**注意：** 需要确保脚本中的输入文件路径、输出文件路径和参数配置正确。由于数据规模较大，脚本运行时间较长(尤其是训练集)。也可自行分批次处理。

In [ ]:
# 注意：脚本默认只取每个数据集的部分语料进行处理作为基线模型的训练数据，参赛选手需根据需求自行修改数据处理策略
# python ./tools/convert_data_to_numerical.py ./tools/spm.model

SyntaxError: invalid syntax (2685084954.py, line 2)

#### OOV

   在encoder-decoder结构中，需要通过固定的词典对平行语料进行表示，**词典大小**一般控制在30k-60k；因此希望**减少词表的大小**，从而提高时间和空间效率。同时还希望**文本长度尽可能的短**，因为文本长度的增加会降低效率并增加神经模型传递信息所需的距离（LSTM），文本越长信息丢失的可能性越大。这就导致了很多**未登录词（OOV）和罕见词（Rare Words）**。
   
   参考[LIC2021-多技能对话赛题冠军方案分享](https://zhuanlan.zhihu.com/p/514079863)的思路，有很多在文本里频繁出现的字符在模型的Vocab文件里没有出现，因此对所有在文本里出现的字符进行统计，将出现次数大于100次的字符保存下来，替换成Vocab文件里一次都没有出现过的字符。
   
   除了一些频繁出现的字符以外，还存在大量的只出现过一次的生僻字符，只用Vocab文件里的字符替换是替换不完的，这些字符的存在会导致模型在infer的时候后续相关的字符也一起乱掉。因此，我们对模型的词表embedding数量进行修改，额外定义了128个特殊字符，对一个样本里的所有字符依次进行遍历，当存在生僻字符时，依次使用特殊字符进行替换。
   

### Step2：构建模型

[UnifiedTransformer](https://github.com/PaddlePaddle/Knover/tree/luge-dialogue/luge-dialogue)以Transformer的编码器为网络基本组件，采用灵活的注意力机制，十分适合文本生成任务，并在模型输入中加入了标识不同对话技能的special token，使得模型能同时支持闲聊对话、推荐对话和知识对话。

**PaddleNLP提供了UnifiedTransformer中文预训练模型，可以通过预训练模型名称完成一键加载。PaddleNLP为了方便用户处理数据，内置了与模型配套的Tokenizer，可以完成文本token化，token转ID，ID转token等操作。**

PaddleNLP目前为UnifiedTransformer提供了两个中文预训练模型：
- `unified_transformer-12L-cn` 该预训练模型是在大规模中文会话数据集上训练得到的
- `unified_transformer-12L-cn-luge` 该预训练模型是`unified_transformer-12L-cn`在千言对话数据集上进行微调得到的

In [3]:
from paddlenlp.transformers import UnifiedTransformerLMHeadModel, UnifiedTransformerTokenizer

# 预训练模型名称
model_name_or_path = 'unified_transformer-12L-cn-luge'

# 加载预训练模型
model = UnifiedTransformerLMHeadModel.from_pretrained(model_name_or_path)
# 加载配套的tokenizer
tokenizer = UnifiedTransformerTokenizer.from_pretrained(model_name_or_path)

[2022-07-19 21:34:09,088] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/unified_transformer-12L-cn-luge/unified_transformer-12L-cn-luge.pdparams
[2022-07-19 21:34:13,755] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/unified_transformer-12L-cn-luge/unified_transformer-12L-cn-vocab.txt
[2022-07-19 21:34:13,759] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/unified_transformer-12L-cn-luge/unified_transformer-12L-cn-spm.model


### 基于预训练模型的模板替换方法
在推荐对话中，我们参考了2020语言与智能技术竞赛推荐对话任务[强行跳大团队的方案](https://zhuanlan.zhihu.com/p/343061563)。使用增加语言模型层的RoBERTa预训练模型，将双向注意力机制改为类似GPT的单向自左向右注意力机制，使得每个Token只能看到它左边的Token。

模型将目标预测以及对话生成集合在一个模型中，并且添加了一个特殊的字符[GOAL]，来表示前一小段文本是下一句话的目标信息。用户和机器人的对话总是包括两部分，Goal 以及对话内容。其中 Goal 总是以[GOAL]作为结尾，表示对话有新目标。大部分情况下，Goal总是为空。User或者Bot 的对话以[SEP]作为结尾。预测时，首先预测回复对应的新目标，如果包含在给定期望目标中，则设置该目标为本轮对话目标，否则本轮对话目标为空。随后生成带有指代字符串的本轮对话回复。

对于对话的输入和生成回复采用了模板填充的方式。对对话中涉及外部信息的部分进行模板替换，模板包括画像信息、知识主体信息、知识客体信息。在预处理时通过规则匹配的方式将对话内容进行模板替换，在后处理时将回复中存在的模板字符串重新替换为外部信息。


### Step3：加载数据

基线通过继承`paddle.io.IterableDataset`自定义可迭代数据集`DialogueDataset`，包括读取文件、shuffle及组batch等操作，细节详见`data.py`。

In [4]:
from paddle.io import DataLoader
from data import DialogueDataset

# 训练batch_size
batch_size = 8192
# 组batch进行排序和shuffle的pool_size
sort_pool_size = 65536

# 训练集路径，注意与数据预处理输出路径保持一致
train_data_path = './datasets/train.txt' 
# 初始化Dataset
train_dataset = DialogueDataset(
        train_data_path,
        batch_size,
        tokenizer.pad_token_id,
        tokenizer.cls_token_id,
        sort_pool_size,
        mode='train')
# 初始化Dataloader
train_dataloader = DataLoader(train_dataset, return_list=True, batch_size=None)

# 开发集路径，注意与数据预处理输出路径保持一致
valid_data_path = './datasets/valid.txt' 
valid_dataset = DialogueDataset(
    valid_data_path,
    batch_size,
    tokenizer.pad_token_id,
    tokenizer.cls_token_id,
    sort_pool_size,
    mode='valid')
valid_dataloader = DataLoader(valid_dataset, return_list=True, batch_size=None)

### Step4：训练优化

在该基线中，我们选择交叉熵损失函数，使用`paddle.optimizer.AdamW`作为优化器。

在训练过程中，模型保存在当前目录checkpoints文件夹下。在训练的同时在验证集上进行评估，输出`loss`和`PPL`等指标。

In [5]:
import os

# 定义训练模型保存函数
def save_ckpt(model, tokenizer, save_dir, name):
    output_dir = os.path.join(save_dir, "model_{}".format(name))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [6]:
import math
import paddle
import paddle.nn.functional as F

# 定义模型评估函数，在模型训练过程中会在开发集上对模型进行评估
@paddle.no_grad()
def evaluation(model, data_loader):
    print('\nEval begin...')
    model.eval()
    total_tokens = 0
    total_loss = 0.0
    start_time = time.time()
    step = 0
    for inputs in data_loader:
        step += 1
        token_ids, type_ids, pos_ids, generation_mask, tgt_label, tgt_pos = inputs

        logits = model(token_ids, type_ids, pos_ids, generation_mask, tgt_pos)
        loss = F.cross_entropy(logits, tgt_label, reduction='sum')

        total_loss += loss.numpy()[0]
        total_tokens += tgt_label.shape[0]

    avg_loss = total_loss / total_tokens
    ppl = math.exp(avg_loss)
    avg_speed = (time.time() - start_time) / step
    print('loss: %.4f - ppl: %.4f - %.3fs/step\n' % (avg_loss, ppl, avg_speed))
    model.train()

In [7]:
import paddle.nn as nn
from paddle.optimizer.lr import NoamDecay
from paddle.optimizer import AdamW

# 学习率
lr = 1e-5
# 学习率逐渐升高到基础学习率（即上面配置的lr）所需要的迭代数
warmup_steps = 4000
# AdamW优化器中使用的weight_decay的系数
weight_decay = 0.01
# 度裁剪允许的最大梯度值
max_grad_norm = 0.1

# 初始化Noam衰减学习率的策略
lr_scheduler = NoamDecay(1 / (warmup_steps * (lr**2)), warmup_steps)
# 对偏置和LayerNorm层不进行weight_decay策略
decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]
# 初始化AdamW优化器
optimizer = AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params,
    grad_clip=nn.ClipGradByGlobalNorm(max_grad_norm))

In [ ]:
import time

# 训练轮次
epochs = 10
# 日志打印间隔
logging_steps = 5
# 模型保存及评估间隔
save_steps = 10
# 模型的保存路径
save_dir = './checkpoints/'

step = 0
total_time = 0.0
for epoch in range(epochs):
    print('\nEpoch %d/%d' % (epoch + 1, epochs))
    batch_start_time = time.time()
    for inputs in train_dataloader:
        step += 1
        token_ids, type_ids, pos_ids, generation_mask, tgt_label, tgt_pos = inputs

        logits = model(token_ids, type_ids, pos_ids, generation_mask, tgt_pos)
        # 使用交叉熵损失函数计算loss
        loss = F.cross_entropy(logits, tgt_label)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()

        total_time += (time.time() - batch_start_time)
        if step % logging_steps == 0:
            ppl = paddle.exp(loss)
            print('step %d - loss: %.4f - ppl: %.4f - lr: %.7f - %.3fs/step'
                % (step, loss, ppl, optimizer.get_lr(), total_time / logging_steps))
            total_time = 0.0
        if step % save_steps == 0:
            # 在开发集上对模型进行评估
            evaluation(model, valid_dataloader)
            # 保存模型
            save_ckpt(model, tokenizer, save_dir, step)
        batch_start_time = time.time()
print("\n=====training complete=====")


Epoch 1/10
step 5 - loss: 4.0267 - ppl: 56.0757 - lr: 0.0000000 - 0.985s/step
step 10 - loss: 3.4819 - ppl: 32.5222 - lr: 0.0000000 - 0.416s/step

Eval begin...
loss: 2.7557 - ppl: 15.7322 - 0.161s/step



[2022-07-19 15:15:57,155] [    INFO] - tokenizer config file saved in ./checkpoints/model_10/tokenizer_config.json
[2022-07-19 15:15:57,158] [    INFO] - Special tokens file saved in ./checkpoints/model_10/special_tokens_map.json


step 15 - loss: 3.5252 - ppl: 33.9609 - lr: 0.0000000 - 0.427s/step
step 20 - loss: 3.5798 - ppl: 35.8652 - lr: 0.0000001 - 0.431s/step

Eval begin...
loss: 2.7557 - ppl: 15.7324 - 0.163s/step



[2022-07-19 15:18:50,562] [    INFO] - tokenizer config file saved in ./checkpoints/model_20/tokenizer_config.json
[2022-07-19 15:18:50,564] [    INFO] - Special tokens file saved in ./checkpoints/model_20/special_tokens_map.json


step 25 - loss: 4.1072 - ppl: 60.7754 - lr: 0.0000001 - 0.425s/step
step 30 - loss: 4.1608 - ppl: 64.1204 - lr: 0.0000001 - 0.421s/step

Eval begin...
loss: 2.7557 - ppl: 15.7327 - 0.163s/step



...

step 305 - loss: 3.6830 - ppl: 39.7666 - lr: 0.0000008 - 0.428s/step
step 310 - loss: 3.4747 - ppl: 32.2892 - lr: 0.0000008 - 0.437s/step

Eval begin...
loss: 2.7574 - ppl: 15.7585 - 0.163s/step



[2022-07-19 16:41:21,942] [    INFO] - tokenizer config file saved in ./checkpoints/model_310/tokenizer_config.json
[2022-07-19 16:41:21,945] [    INFO] - Special tokens file saved in ./checkpoints/model_310/special_tokens_map.json


step 315 - loss: 3.7047 - ppl: 40.6366 - lr: 0.0000008 - 0.442s/step
step 320 - loss: 3.3017 - ppl: 27.1601 - lr: 0.0000008 - 0.433s/step

Eval begin...
loss: 2.7574 - ppl: 15.7595 - 0.163s/step



[2022-07-19 16:44:11,912] [    INFO] - tokenizer config file saved in ./checkpoints/model_320/tokenizer_config.json
[2022-07-19 16:44:11,915] [    INFO] - Special tokens file saved in ./checkpoints/model_320/special_tokens_map.json


step 325 - loss: 3.4832 - ppl: 32.5641 - lr: 0.0000008 - 0.430s/step
step 330 - loss: 3.4689 - ppl: 32.1020 - lr: 0.0000008 - 1.415s/step

Eval begin...


### Step5：预测解码

用训练保存的模型参数来初始化模型，加载测试集后即可进行预测。

**PaddleNLP针对生成式任务提供了`generate`函数，支持Greedy Search、Beam Search和Sampling解码策略，用户只需指定解码策略以及相应的参数即可完成预测解码，得到生成的sequence的token ids以及概率得分。**

In [8]:
%pwd

'/home/aistudio/multi-skill_dialogue'

In [9]:
# 这里可以是paddlenlp提供的预训练模型名称，或者自己训练获得的微调模型路径
model_name_or_path = './checkpoints/model_320' 
# 加载模型
model = UnifiedTransformerLMHeadModel.from_pretrained(model_name_or_path)

In [12]:
import time

In [13]:
# 预测batch_size
batch_size = 4

# 测试集路径，注意与数据预处理输出路径保持一致
test_data_path = './datasets/test.txt' 
test_dataset = DialogueDataset(
    test_data_path,
    batch_size,
    tokenizer.pad_token_id,
    tokenizer.cls_token_id,
    mode='test')
test_dataloader = DataLoader(test_dataset, return_list=True, batch_size=None)

In [11]:
from data import select_response

# 预测解码生成序列的最大长度
max_dec_len = 64
# 预测解码生成序列的最小长度
min_dec_len = 1
# 解码策略
decode_strategy = 'sampling'
# topk-sampling解码参数top_k
top_k = 5
# 每条输入序列返回的输出序列个数，生成式API内部会将输入序列进行复制
num_return_sequences = 20
# 文本结果序列保存路径
output_path = './predict.txt'
# 日志打印间隔
logging_steps = 1

print('\nInfer begin...')
model.eval()
total_time = 0.0
start_time = time.time()
responses = []
for step, inputs in enumerate(test_dataloader, 1):
    input_ids, token_type_ids, position_ids, attention_mask = inputs
    ids, scores = model.generate(
        input_ids=input_ids,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        attention_mask=attention_mask,
        max_length=max_dec_len,
        min_length=min_dec_len,
        decode_strategy=decode_strategy,
        top_k=top_k,
        num_return_sequences=num_return_sequences)

    total_time += (time.time() - start_time)
    if step % logging_steps == 0:
        print('step %d - %.3fs/step' % (step, total_time / logging_steps))
        total_time = 0.0
    # 模型输出序列排序，从num_return_sequences个序列中选出最好的一个作为结果
    results = select_response(ids, scores, tokenizer, max_dec_len, num_return_sequences)
    responses.extend(results)

    start_time = time.time()

# 保存文本结果序列
with open(output_path, 'w', encoding='utf-8') as fout:
    for response in responses:
        fout.write(response + '\n')
print('\nSave inference result into: %s' % output_path)


Infer begin...
step 1 - 2.609s/step
step 2 - 2.460s/step
step 3 - 2.469s/step
step 4 - 2.359s/step
step 5 - 2.682s/step
step 6 - 2.744s/step
step 7 - 2.373s/step
step 8 - 2.311s/step


### 快速搭建基线Step6：提交结果

预测结果会被保存在`output_path`中，将预测结果准备成比赛官网要求的格式提交。